**Final Code**

Installing faiss


In [ ]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Imports and device settings

In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from matplotlib import pyplot as plt
%matplotlib inline

# Let's first define our device as the first visible cuda device if we have
# CUDA available:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


Loading data as training and testing sets

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


Using Dataloader to load batched data with applied transformation

In [ ]:
batch_size = 10

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Plotting Few images in dataset

In [ ]:
for batch in trainloader:
    inputs, targets = batch
    for img in inputs:
        image  = img.cpu().numpy()
        # transpose image to fit plt input
        image = image.T
        image=image.transpose(1,0,2)#1,0,2
        # normalise image
        data_min = np.min(image, axis=(0,1), keepdims=True)
        data_max = np.max(image, axis=(0,1), keepdims=True)
        scaled_data = (image - data_min) / (data_max - data_min)
        # show image
        plt.imshow(scaled_data)
        plt.show()
        break


Using pretrained RESNET18 model for extracting features from images &
Obtaining Image Embeddings

In [ ]:
model_a= torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# Select the desired layer
layer_a = model_a._modules.get('avgpool')

def image_embeddings(m, i, o):
    """Copy embeddings from the penultimate layer.
    """
    o = o[:, :, 0, 0].detach().numpy().tolist()
    outputs_a.append(o)

outputs_a = []
# attach hook to the penulimate layer
_ = layer_a.register_forward_hook(image_embeddings)

model_a.eval() # Inference model

# Generate image's embeddings for all images in dloader and saves
# them in the list outputs
# count=0
for i in trainloader:
    _ = model_a(i[0])
    # count+=1
    # print(count)
# flatten list of embeddings to remove batches
list_embeddings1 =np.array([np.float32(item) for sublist in outputs_a for item in sublist])

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

IndexFlatL2

In [ ]:
%%time
d=list_embeddings1.shape[1]
import faiss
#IndexFlatL2
index_table1 = faiss.IndexFlatL2(d)
index_table1.add(list_embeddings1)

CPU times: user 61.4 ms, sys: 47 ms, total: 108 ms
Wall time: 124 ms


In [ ]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
index_table1.is_trained

True

In [ ]:
# print(type(list_embeddings[0][0]))
# print(list_embeddings.shape)
# index_table1.add(list_embeddings1)

Checking embeddings are added or not

In [ ]:
index_table1.ntotal

50000

Generating embeddings for test set- query embeddings

In [ ]:
outputs=[]

def image_embeddings(m, i, o):
    """Copy embeddings from the penultimate layer.
    """
    o = o[:, :, 0, 0].detach().numpy().tolist()
    outputs.append(o)
def create_img_embeddings():
    outputs = []
# attach hook to the penulimate layer
    _ = layer.register_forward_hook(image_embeddings)
    model_a.eval() # Inference model

# Generate image's embeddings for all images in dloader and saves
# them in the list outputs
# count=0
for i in testloader:
    model_a = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
    # Select the desired layer
    layer = model_a._modules.get('avgpool')
    create_img_embeddings()
    _ = model_a(i[0])
    # count+=1
    # print(count)
# flatten list of embeddings to remove batches
query_embeddings1 =np.array([np.float32(item) for sublist in outputs for item in sublist])
print(query_embeddings1.shape)





Print images corresponding to IDs

In [ ]:
def print_images(I):
  ID1=I[0][0]
  ID2=I[0][1]
  ID3=I[0][2]
  ID4=I[0][3]
  ID5=I[0][4]
  rows=0
  cols=1
  for batch in trainloader:
      inputs, targets = batch
      for img in inputs:
        if(rows*10+cols == ID1 or rows*10+cols == ID2 or rows*10+cols == ID3 or rows*10+cols == ID4 or rows*10+cols == ID5):
            image  = img.cpu().numpy()
            # transpose image to fit plt input
            image = image.T
            image=image.transpose(1,0,2)
            # normalise image
            data_min = np.min(image, axis=(0,1), keepdims=True)
            data_max = np.max(image, axis=(0,1), keepdims=True)
            scaled_data = (image - data_min) / (data_max - data_min)
            # show image
            plt.imshow(scaled_data)
            plt.show()
        cols+=1
      rows+=1
      cols=1





Printing similar 5 IDs corresponding to query images & printing them

In [ ]:
%%time
k=5
count=1
for i in query_embeddings1:
    j=i.reshape(1,512)
    D,I = index_table1.search(j,k)
    print('similar 5 IDs corresponding to query-',count)
    count=count+1
    print(D)
    print(I)
    # print_images(I)
    # break

**Using modified pretrianed RESNET18 model for obtaining image embeddings**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AvgPool2d(4)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def _resnet(block, layers):
    model = ResNet(block, layers)
    return model

def ResNet18():
    return _resnet(BasicBlock, [2, 2, 2, 2])


In [ ]:
model_b=ResNet18()

In [ ]:
# Select the desired layer
layer_b = model_b._modules.get('avgpool')

def image_embeddings(m, i, o):
    """Copy embeddings from the penultimate layer.
    """
    o = o[:, :, 0, 0].detach().numpy().tolist()
    outputs.append(o)

outputs = []
# attach hook to the penulimate layer
_ = layer_b.register_forward_hook(image_embeddings)

model_b.eval() # Inference model

# Generate image's embeddings for all images in dloader and saves
# them in the list outputs
# count=0
for i in trainloader:
    _ = model_b(i[0])
    # count+=1
    # print(count)
# flatten list of embeddings to remove batches
list_embeddings2 =np.array([np.float32(item) for sublist in outputs for item in sublist])

In [ ]:
print(list_embeddings2.shape)

(50000, 512)


In [ ]:
%%time
d=list_embeddings2.shape[1]
import faiss
#IndexFlatL2
index_table2 = faiss.IndexFlatL2(d)
index_table2.add(list_embeddings2)

CPU times: user 28 ms, sys: 46 ms, total: 74 ms
Wall time: 74 ms


In [ ]:
index_table2.is_trained

True

In [ ]:
# index_table2.add(list_embeddings2)

In [ ]:
print(index_table2.ntotal)

50000


In [ ]:
outputs=[]

def image_embeddings(m, i, o):
    """Copy embeddings from the penultimate layer.
    """
    o = o[:, :, 0, 0].detach().numpy().tolist()
    outputs.append(o)
def create_img_embeddings():
    outputs = []
# attach hook to the penulimate layer
    _ = layer.register_forward_hook(image_embeddings)
    model_b.eval() # Inference model

# Generate image's embeddings for all images in dloader and saves
# them in the list outputs
# count=0
for i in testloader:
    model_b = ResNet18()
    # Select the desired layer
    layer = model_b._modules.get('avgpool')
    create_img_embeddings()
    _ = model_b(i[0])
    # count+=1
    # print(count)
# flatten list of embeddings to remove batches
query_embeddings2 =np.array([np.float32(item) for sublist in outputs for item in sublist])
print(query_embeddings2.shape)





(10000, 512)


Printing similar 5 IDs corresponding to query images & printing them

In [ ]:
%%time
# def print_images(I):
#   ID1=I[0][0]
#   ID2=I[0][1]
#   ID3=I[0][2]
#   ID4=I[0][3]
#   ID5=I[0][4]
#   # print(ID1)
#   rows=0
#   cols=1
#   for batch in trainloader:
#       inputs, targets = batch
#       for img in inputs:
#         # print(rows*10+cols)
#         if(rows*10+cols == ID1 or rows*10+cols == ID2 or rows*10+cols == ID3 or rows*10+cols == ID4 or rows*10+cols == ID5):
#             image  = img.cpu().numpy()
#             # transpose image to fit plt input
#             image = image.T
#             image=image.transpose(1,0,2)
#             # normalise image
#             data_min = np.min(image, axis=(0,1), keepdims=True)
#             data_max = np.max(image, axis=(0,1), keepdims=True)
#             scaled_data = (image - data_min) / (data_max - data_min)
#             # show image
#             plt.imshow(scaled_data)
#             plt.show()
#         cols+=1
#       rows+=1
#       cols=1

k=5
count=1

for i in query_embeddings2:
    j=i.reshape(1,512)
    D,I = index_table2.search(j,k)
    print('similar 5 IDs corresponding to query-',count)
    count=count+1
    print(I)
    # print_images(I)
    break

similar 5 IDs corresponding to query- 1
[[14940 14171   518  8843 32577]]
CPU times: user 38.2 ms, sys: 976 µs, total: 39.1 ms
Wall time: 38.5 ms


**Using Qunatisers**

**Using IVFFlat Quantiser**

In [ ]:
%%time
#Using IndexIVFFlat quantiser - q1
nlist=50
quantiser=faiss.IndexFlatL2(d)
# index_table1_q1=faiss.IndexIVFFlat(quantiser,d,nlist)

# index_table1_q1.train(list_embeddings1)

# index_table1_q1.add(list_embeddings1)


CPU times: user 21 µs, sys: 1e+03 ns, total: 22 µs
Wall time: 24.1 µs


In [ ]:
%%time
index_table1_q1=faiss.IndexIVFFlat(quantiser,d,nlist)

index_table1_q1.train(list_embeddings1)

index_table1_q1.add(list_embeddings1)

CPU times: user 502 ms, sys: 13 ms, total: 515 ms
Wall time: 515 ms


In [ ]:
%%time
index_table2_q1=faiss.IndexIVFFlat(quantiser,d,nlist)
index_table2_q1.train(list_embeddings2)
index_table2_q1.add(list_embeddings2)

CPU times: user 179 ms, sys: 7.01 ms, total: 186 ms
Wall time: 181 ms


Checking whether index tables are trained or not

In [ ]:
index_table1_q1.is_trained
index_table2_q1.is_trained

True

Training index tables

In [ ]:
# index_table1_q1.train(list_embeddings1)
# index_table2_q1.train(list_embeddings2)

Checking whether index tables are trained or not

In [ ]:
index_table1_q1.is_trained
index_table2_q1.is_trained

True

Adding image embeddings to index tables

In [ ]:
# index_table1_q1.add(list_embeddings1)
# index_table2_q1.add(list_embeddings2)

Testing IVFFlat qunatiser for table 1

In [ ]:
%%time
#testing index_table_q1
k=5
count=1

for i in query_embeddings1:
    j=i.reshape(1,512)
    D,I = index_table1_q1.search(j,k)
    print('similar 5 IDs corresponding to query-',count)
    count=count+1
    print(I)
    # print_images(I)
    break

similar 5 IDs corresponding to query- 1
[[ 7695  3834 34354  2049   944]]
CPU times: user 1.97 ms, sys: 0 ns, total: 1.97 ms
Wall time: 2.49 ms


Testing IVFFlat quantiser for table 2

In [ ]:
%%time
#testing index_table_q2
k=5
count=1

for i in query_embeddings2:
    j=i.reshape(1,512)
    D,I = index_table2_q1.search(j,k)
    print('similar 5 IDs corresponding to query-',count)
    count=count+1
    print(I)
    # print_images(I)
    break

similar 5 IDs corresponding to query- 1
[[14940 14171   518  8843 32577]]
CPU times: user 37.4 ms, sys: 0 ns, total: 37.4 ms
Wall time: 36.3 ms


**Using IVFPQ Quantiser**

In [ ]:
%%time
#Using IndexIVFPQ quantiser - q2
m=8
bits=8

quantiser2=faiss.IndexFlatL2(d)


CPU times: user 28 s, sys: 235 ms, total: 28.3 s
Wall time: 28.1 s


In [ ]:
%%time
index_table1_q2=faiss.IndexIVFPQ(quantiser2,d,nlist,m,bits)

index_table1_q2.train(list_embeddings1)

index_table1_q2.add(list_embeddings1)


CPU times: user 14.2 s, sys: 66.7 ms, total: 14.3 s
Wall time: 14.2 s


In [ ]:
%%time
index_table2_q2=faiss.IndexIVFPQ(quantiser2,d,nlist,m,bits)
index_table2_q2.train(list_embeddings2)
index_table2_q2.add(list_embeddings2)

CPU times: user 14.1 s, sys: 62.6 ms, total: 14.1 s
Wall time: 14.1 s


Checking whether index tables are trained or not

In [ ]:
index_table1_q2.is_trained
index_table2_q2.is_trained

True

Training index tables

In [ ]:
# index_table1_q2.train(list_embeddings1)
# index_table2_q2.train(list_embeddings2)

checking whether index tables are trained or not

In [ ]:
index_table1_q2.is_trained
index_table2_q2.is_trained

True

Adding image embeddings to index tables

In [ ]:
# index_table1_q2.add(list_embeddings1)
# index_table2_q2.add(list_embeddings2)

IVFPQ testing for index table 1

In [ ]:
%%time
#testing index_table_q1
k=5
count=1

for i in query_embeddings1:
    j=i.reshape(1,512)
    D,I = index_table1_q2.search(j,k)
    print('similar 5 IDs corresponding to query-',count)
    count=count+1
    print(I)
    # print_images(I)
    break

similar 5 IDs corresponding to query- 1
[[40462  3834 40950  7695 34751]]
CPU times: user 593 µs, sys: 0 ns, total: 593 µs
Wall time: 609 µs


IVFPQ testing for index table 2

In [ ]:
%%time
#testing index_table_q2
k=5
count=1

for i in query_embeddings2:
    j=i.reshape(1,512)
    D,I = index_table2_q2.search(j,k)
    print('similar 5 IDs corresponding to query-',count)
    count=count+1
    print(I)
    print_images(I)


NameError: ignored